# wps_CI

#### wps_CI is a process that runs the [ci.netcdf.wrapper](https://github.com/pacificclimate/ClimDown/blob/master/R/CI.R#L235) function of [ClimDown](https://cran.r-project.org/web/packages/ClimDown/index.html) package. To get started, first instatiate the client. Here, the client will try to connect to a remote chickadee instance using the url parameter.

In [1]:
import os
from birdy import WPSClient
from wps_tools.testing import get_target_url
from pkg_resources import resource_filename
from wps_tools.utils import copy_http_content
from tempfile import NamedTemporaryFile
from netCDF4 import Dataset

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "chickadee":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("chickadee")
print(f"Using chickadee on {url}")

Using chickadee on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/chickadee/wps


In [3]:
chickadee = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [4]:
# NBVAL_IGNORE_OUTPUT
chickadee.ci?

Signature:
chickadee.ci(
    gcm_file,
    obs_file=None,
    varname=None,
    loglevel='INFO',
    out_file=None,
)
Docstring:
Climate Imprint (CI) downscaling

Parameters
----------
gcm_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of GCM simulations
obs_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of high-res gridded historical observations
varname : string
    Name of the NetCDF variable to downscale (e.g. 'tasmax')
out_file : string
    Filename to create with the climate imprint outputs
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
output : ComplexData:mimetype:`application/x-netcdf`
    output netCDF file
File:      ~/Desktop/chickadee/</home/sangwonl/Desktop/chickadee/venv/lib/python3.7/site-packages/birdy/client/base.py-0>
Type:      method


#### We can use the docstring to ensure we provide the appropriate parameters.

In [5]:
with NamedTemporaryFile(suffix=".nc", prefix="output_", dir="/tmp", delete=True) as out_file:
    output = chickadee.ci(
        gcm_file = resource_filename("tests", "/data/tiny_gcm.nc"),
        obs_file = resource_filename("tests", "/data/tiny_obs.nc"),
        out_file = out_file.name,
        varname = "tasmax",
    )

#### Once the process has completed we can extract the results and ensure it is what we expected.

In [6]:
with NamedTemporaryFile(suffix=".nc", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
    output_data = Dataset(copy_http_content(output.get()[0], tmp_file))
    expected_data = Dataset(resource_filename("tests", "/data/CI_expected_output.nc"))
    for key, value in expected_data.dimensions.items():
        assert str(output_data.dimensions[key]) == str(value)